resnet18  
fcn

In [99]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

# 데이터셋 크기에 맞는 CNN 모델 불러오기 (예: ResNet18)
model1 = models.resnet18(pretrained=True)

# 모델의 마지막 레이어를 새로운 fully connected layer로 교체
num_features = model1.fc.in_features
model1.fc = nn.Linear(num_features, 2)  # 예시에서는 클래스가 2개라 가정

# GPU 사용이 가능하다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.to(device)

# Loss function 및 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

x_np = X.to_numpy()
y_np = y.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x_np, y_np, test_size= 0.1, shuffle=True)

c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [66]:
len(y_train), len(y_test)

(7200, 800)

In [100]:
def change_tensor(data):
    np_test_tensor = []
    for i in data:
        np_test = np.tile(i, (1,4))
        np_test = np.tile(np_test, (28,1))
        np_test = torch.tensor(np_test, dtype=torch.float32)
        np_test = np_test.expand(3,-1,-1)

        np_test_tensor.append(np_test)

    np_test_tensor = np.array(np_test_tensor)
    np_test_tensor = torch.tensor(np_test_tensor)
    return np_test_tensor

In [101]:
x_train = change_tensor(x_train)
x_test = change_tensor(x_test)
y_train = torch.tensor(np.tile(y_train, (1,)), dtype=torch.long)
y_test = torch.tensor(np.tile(y_test, (1,)), dtype=torch.long)

In [102]:
x_train.shape

torch.Size([7200, 3, 28, 28])

In [103]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data
        label = self.labels
            
        return data[idx], label[idx]

In [104]:
train_dataset = CustomDataset(x_train, y_train)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [105]:
# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model1.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(device)  # 배치 차원 추가
        outputs = model1(inputs)
        loss = criterion(outputs, labels.to(device))  # 배치 차원 추가
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(x_train)}")

print("학습 완료")

Epoch [1/10], Loss: 0.006942493869638485
Epoch [2/10], Loss: 0.003655602261165364
Epoch [3/10], Loss: 0.002698197704093117
Epoch [4/10], Loss: 0.002555801309709851
Epoch [5/10], Loss: 0.001990881750955143
Epoch [6/10], Loss: 0.0016875280388113526
Epoch [7/10], Loss: 0.0016304093971848488
Epoch [8/10], Loss: 0.00172927631809014
Epoch [9/10], Loss: 0.001512985364372273
Epoch [10/10], Loss: 0.0013260644622399317
학습 완료


In [106]:
correct = 0
total = 0

# 모델을 evaluation 모드로 설정
model1.eval()

# 모델이 예측한 결과와 실제 레이블을 비교하여 정확도 계산
with torch.no_grad():
    for inputs, labels in zip(x_test, y_test):
        inputs = inputs.unsqueeze(0).to(device)
        outputs = model1(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels.to(device)).sum().item()

accuracy = correct / total
print(accuracy)
print(f"Accuracy: {accuracy * 100:.4f}%")


0.97
Accuracy: 97.0000%


resnet18  
1d conv layer

In [94]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pickle
import numpy as np

def change_tensor(data):
    np_test_tensor = []
    for i in data:
        np_test = np.tile(i, (1,4))
        np_test = np.tile(np_test, (28,1))
        np_test = torch.tensor(np_test, dtype=torch.float32)
        np_test = np_test.expand(3,-1,-1)

        np_test_tensor.append(np_test)

    np_test_tensor = np.array(np_test_tensor)
    np_test_tensor = torch.tensor(np_test_tensor)
    return np_test_tensor

# 데이터셋 크기에 맞는 CNN 모델 불러오기 (예: ResNet18)
model2 = models.resnet18(pretrained=True)

# 모델의 마지막 레이어를 변경하고자 할 경우
# 모델의 마지막 레이어 이전의 출력 차원을 확인
num_features = model2.fc.in_features

# 새로운 모델 정의
class CustomModel(nn.Module):
    def __init__(self, num_features):
        super(CustomModel, self).__init__()
        self.features = nn.Sequential(*list(model2.children())[:-2])  # ResNet의 마지막 2개 레이어 제외
        self.conv1d = nn.Conv1d(num_features, 64, kernel_size=3, stride=1, padding=1)  # 1D Convolutional Layer 추가
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, 2)  # 예시에서는 클래스가 2개라 가정

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), num_features, -1)  # Flatten 작업
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # 1D로 평탄화
        x = self.fc(x)
        return x

# 새로운 모델 생성
model2 = CustomModel(num_features)

# GPU 사용이 가능하다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = model2.to(device)

# Loss function 및 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

x_np = X.to_numpy()
y_np = y.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x_np, y_np, test_size= 0.1, shuffle=True)

x_train = change_tensor(x_train)
x_test = change_tensor(x_test)
y_train = torch.tensor(np.tile(y_train, (1,)), dtype=torch.long)
y_test = torch.tensor(np.tile(y_test, (1,)), dtype=torch.long)

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data
        label = self.labels
            
        return data[idx], label[idx]

train_dataset = CustomDataset(x_train, y_train)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(device)  # 배치 차원 추가
        outputs = model2(inputs)
        loss = criterion(outputs, labels.to(device))  # 배치 차원 추가
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(x_train)}")

print("학습 완료")


c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.008073243932125882
Epoch [2/10], Loss: 0.003722975878044963
Epoch [3/10], Loss: 0.0027511982281511237
Epoch [4/10], Loss: 0.0027145783400111313
Epoch [5/10], Loss: 0.0020111356867710127
Epoch [6/10], Loss: 0.0019502820013116838
Epoch [7/10], Loss: 0.0018316858849074277
Epoch [8/10], Loss: 0.0015834826717683527
Epoch [9/10], Loss: 0.0014597541671052265
Epoch [10/10], Loss: 0.0014379769831430167
학습 완료


In [1]:
correct = 0
total = 0

# 모델을 evaluation 모드로 설정
model2.eval()

# 모델이 예측한 결과와 실제 레이블을 비교하여 정확도 계산
with torch.no_grad():
    for inputs, labels in zip(x_test, y_test):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(accuracy)
print(f"Accuracy: {accuracy * 100:.2f}%")


NameError: name 'model2' is not defined

resnet18  
2d conv layer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pickle
import numpy as np

def change_tensor(data):
    np_test_tensor = []
    for i in data:
        np_test = np.tile(i, (1,4))
        np_test = np.tile(np_test, (28,1))
        np_test = torch.tensor(np_test, dtype=torch.float32)
        np_test = np_test.expand(3,-1,-1)

        np_test_tensor.append(np_test)

    np_test_tensor = np.array(np_test_tensor)
    np_test_tensor = torch.tensor(np_test_tensor)
    return np_test_tensor

# 데이터셋 크기에 맞는 CNN 모델 불러오기 (예: ResNet18)
model3 = models.resnet18(pretrained=True)

# 모델의 마지막 레이어를 변경하고자 할 경우
# 모델의 마지막 레이어 이전의 출력 차원을 확인
num_features = model3.fc.in_features

# 새로운 모델 정의
class CustomModel(nn.Module):
    def __init__(self, num_features):
        super(CustomModel, self).__init__()
        self.features = nn.Sequential(*list(model3.children())[:-2])  # ResNet의 마지막 2개 레이어 제외
        self.conv2d = nn.Conv2d(num_features, 64, kernel_size=3, stride=1, padding=1)  # 1D Convolutional Layer 추가
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(64, 2)  # 예시에서는 클래스가 2개라 가정

    def forward(self, x):
        x = self.features(x)
        x = self.conv2d(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # 2D 출력을 1D로 평탄화
        x = self.fc(x)
        return x

# 새로운 모델 생성
model3 = CustomModel(num_features)

# GPU 사용이 가능하다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3 = model3.to(device)

# Loss function 및 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

x_np = X.to_numpy()
y_np = y.to_numpy()

x_train, x_test, y_train, y_test = train_test_split(x_np, y_np, test_size= 0.1, shuffle=True)

x_train = change_tensor(x_train)
x_test = change_tensor(x_test)
y_train = torch.tensor(np.tile(y_train, (1,)), dtype=torch.long)
y_test = torch.tensor(np.tile(y_test, (1,)), dtype=torch.long)

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data
        label = self.labels
            
        return data[idx], label[idx]

train_dataset = CustomDataset(x_train, y_train)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model3.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.to(device)  # 배치 차원 추가
        outputs = model3(inputs)
        loss = criterion(outputs, labels.to(device))  # 배치 차원 추가
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(x_train)}")

print("학습 완료")


In [7]:
correct = 0
total = 0

# 모델을 evaluation 모드로 설정
model3.eval()

# 모델이 예측한 결과와 실제 레이블을 비교하여 정확도 계산
with torch.no_grad():
    for inputs, labels in zip(x_train, y_train):
        inputs = inputs.unsqueeze(0).to(device)
        outputs = model3(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels.to(device)).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%


In [4]:
import pickle

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

In [5]:
X

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity
0,-1.924968,0.468078,3.077832,-1.472177,0.294799,2.435570,0.271290
1,-2.409751,0.486870,0.346921,-2.495099,-0.892213,2.067549,0.307325
2,-0.357607,1.483176,1.568452,-2.645145,-0.647267,3.090643,1.427322
3,-0.868524,1.566201,1.889605,-1.273761,-1.006278,1.873001,0.477862
4,0.651825,1.319199,-0.022459,-1.209709,-1.430692,1.078345,2.812442
...,...,...,...,...,...,...,...
7995,-6.414403,0.723565,1.134953,2.952763,0.297928,-0.156946,2.398091
7996,0.851143,-2.217875,-2.812175,0.489249,-1.323410,-2.316883,2.113136
7997,1.422722,-1.907665,-2.532364,0.964976,-0.562375,-1.834765,0.697361
7998,-2.131904,-2.742600,-1.008029,2.126946,-0.802632,-3.580266,0.423569


In [9]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# MinMaxScaler를 사용하여 데이터프레임의 값을 -1에서 1 사이의 값으로 변환
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [10]:
normalized_X

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity
0,-0.239382,0.253494,0.361423,-0.278049,0.134624,0.343866,0.086832
1,-0.300098,0.256186,-0.029450,-0.412636,-0.036622,0.293700,0.091440
2,-0.043080,0.398896,0.145387,-0.432378,-0.001284,0.433160,0.234675
3,-0.107069,0.410788,0.191353,-0.251943,-0.053077,0.267181,0.113250
4,0.083345,0.375408,-0.082319,-0.243516,-0.114306,0.158859,0.411816
...,...,...,...,...,...,...,...
7995,-0.801655,0.290090,0.083341,0.304146,0.135075,-0.009526,0.358825
7996,0.108308,-0.131238,-0.481608,-0.019982,-0.098829,-0.303952,0.322383
7997,0.179895,-0.086804,-0.441559,0.042611,0.010963,-0.238234,0.141321
7998,-0.265300,-0.206399,-0.223382,0.195492,-0.023698,-0.476167,0.106306


In [11]:
normalized_X.to_pickle('xTrain_normalized.pkl')